# Market Simulator
ML for trading Udacity Course exercise

More info:
https://quantsoftware.gatech.edu/Marketsim

A transcription of the Udacity Course lectures can be find on https://docs.google.com/document/d/1ELqlnuTSdc9-MDHOkV0uvSY4RmI1eslyQlU9DgOY_jc/edit?usp=sharing

Kairoart 2018



## Overview

In this project you will create a market simulator that accepts trading orders and keeps track of a portfolio's value over time and then assesses the performance of that portfolio. 

## Part 1: Create a market simulation function

Your job is to implement your market simulator as a function, compute_portvals() that returns a dataframe with one column. It should adhere to the following API: 

    def compute_portvals(orders_file = "./orders/orders.csv", start_val = 1000000, commission = 9.95, impact = 0.005):
        # TODO: Your code here
        return portvals

The start date and end date of the simulation are the first and last dates with orders in the orders_file. The arguments are as follows:

    orders_file is the name of a file from which to read orders, and
    start_val is the starting value of the portfolio (initial cash available)
    commission is the fixed amount in dollars charged for each transaction (both entry and exit)
    impact is the amount the price moves against the trader compared to the historical data at each transaction

Return the result (portvals) as a single-column pandas.DataFrame (column name does not matter), containing the value of the portfolio for each trading day in the first column from start_date to end_date, inclusive.

The files containing orders are CSV files with the following columns:

    Date (yyyy-mm-dd)
    Symbol (e.g. AAPL, GOOG)
    Order (BUY or SELL)
    Shares (no. of shares to trade)

For example: 

    Date,Symbol,Order,Shares
    2008-12-3,AAPL,BUY,130
    2008-12-8,AAPL,SELL,130
    2008-12-5,IBM,BUY,50


### Goal

Your simulator should calculate the total value of the portfolio for each day using adjusted closing prices (cash plus value of equities) and print the result to the file values.csv. The contents of the values.csv file should look something like this:

    2008, 12, 3, 1000000
    2008, 12, 4, 1000010
    2008, 12, 5, 1000250
    ...


### How it should work

Your code should keep account of how many shares of each stock are in the portfolio on each day and how much cash is available on each day. Note that negative shares and negative cash are possible. Negative shares mean that the portfolio is in a short position for that stock. Negative cash means that you've borrowed money from the broker.

When a BUY order occurs, you should add the appropriate number of shares to the count for that stock and subtract the appropriate cost of the shares from the cash account. The cost should be determined using the adjusted close price for that stock on that day.

When a SELL order occurs, it works in reverse: You should subtract the number of shares from the count and add to the cash account. 

### Report

    * Plot the price history over the trading period.
    * Sharpe ratio (Always assume you have 252 trading days in an year. And risk free rate = 0) of the total portfolio
    * Cumulative return of the total portfolio
    * Standard deviation of daily returns of the total portfolio
    * Average daily return of the total portfolio
    * Ending value of the portfolio

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import operator
import sys
import csv
import math

# To fetch data
from pandas_datareader import data as pdr   
import fix_yahoo_finance as yf  
yf.pdr_override()   


### Read orders from file

In [2]:
def readOrdersFileIntoDF(filename):

    # opening the filename
    fr = open(filename)
    
    # for row count in 
    index=0
    
    # Lists used for making the dataframe.
    dtList = []
    symbolList = []
    orderTypeList = []
    volumeList = []
    
    # For each line
    # A Sample Line - 2011,1,14,AAPL,Buy,1500
    for orderString in fr.readlines():

        # Stripping off the return line character
        orderString=orderString.strip()
        
        # Splitting the line and getting a List back
        listFromLine = orderString.split(',')
        
        # Adding the dates into dtList. 16,00,00 for 1600 hrs
        dtList.append(dt.datetime(int(listFromLine[0]), int(listFromLine[1]), int(listFromLine[2])))
        
        # Adding the symbols into symbolList
        symbolList.append(listFromLine[3])
        
        # Adding the orders into orderTypeList
        orderTypeList.append(listFromLine[4])
        
        # Adding the number of shares into volumeList
        volumeList.append(listFromLine[5])

    # Creating a Dictionary for converting it into DataFrame later
    data = { 'datetime' : dtList, 'symbol' : symbolList, 'ordertype':orderTypeList, 'volume':volumeList }

    # Converting the Dictinary into a nice looking Pandas Dataframe ordered by datetime index
    ordersDataFrame = pd.DataFrame(data)
    ordersDataFrame.index= ordersDataFrame['datetime']
    
    # Drop datetime column
    ordersDataFrame.drop('datetime', axis=1, inplace=True)
    #print(ordersDataFrame)
    
    
    # Getting the Symbols from the Orders. This list will be required for fetching the prices
    symbolList = list(set(ordersDataFrame['symbol']))
    
    # Returning it.
    return ordersDataFrame, symbolList

### Get data from Yahoo for the given dates 

In [3]:
def fetchData(dt_start, dt_end, ls_symbols):

    # Add a day to dt_end for Yahoo purpose
    dt_end = pd.to_datetime(dt_end) + pd.DateOffset(days=1)
    
    # Get data of trading days between the start and the end.
    df = pdr.get_data_yahoo(
            # tickers list (single tickers accepts a string as well)
            tickers = ls_symbols,

            # start date (YYYY-MM-DD / datetime.datetime object)
            # (optional, defaults is 1950-01-01)
            start = dt_start,

            # end date (YYYY-MM-DD / datetime.datetime object)
            # (optional, defaults is Today)
            end = dt_end,

            # return a multi-index dataframe
            # (optional, default is Panel, which is deprecated)
            as_panel = False,

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            group_by = 'ticker',

            # adjust all OHLC automatically
            # (optional, default is False)
            auto_adjust = False
    )
        

    # Getting the numpy ndarray of Adj Close prices.
    adj_close_price = df.loc[:, (slice(None), ('Adj Close'))]

    
    # returning the Adj Closed prices for all the days    
    return adj_close_price

### Market simulator

In [4]:
def marketsim(initialCash, ordersdf, symbols):

    # reading the boundary dates
    dt_start = ordersdf.index[0]
    dt_end = ordersdf.index[len(ordersdf)-1]
    
    # All the adjustedClosingPrices fetched from NYSE within the range and for given symbols
    closingPrices = fetchData(dt_start, dt_end, symbols)
    num_tradingDays = len(closingPrices)
    #print(closingPrices)
    
    #Cash for the days
    temp = np.zeros((num_tradingDays, 1))
    cash = pd.DataFrame(temp, columns = ['cashinhand'])
    
    #Value for the days
    temp = np.zeros((num_tradingDays, 1))
    valueFrame = pd.DataFrame(temp, columns = ['valueOfPortfolio'])

    #Setting the first value to be the initial cash amount.
    cash.cashinhand.loc[0] = initialCash
    
    index = 0
    
    for tradingDayIndex in range(num_tradingDays):  
        if tradingDayIndex != 0:
            cash.cashinhand.iloc[tradingDayIndex] = cash.cashinhand.iloc[tradingDayIndex - 1] 
        else:
            cash.cashinhand.iloc[tradingDayIndex] = initialCash
            
        for tradingOrder in ordersDataFrame.index:
            if tradingOrder == closingPrices.index[tradingDayIndex]:
                if ordersdf.ordertype.iloc[index] == 'Buy':
                    toBuySymbol = ordersdf.symbol.iloc[index]
                    toBuy = symbols.index(toBuySymbol)
                    numShares = ordersdf.volume.iloc[index]
                    priceForTheDay = closingPrices.iloc[tradingDayIndex][toBuy]
                    cash.cashinhand.iloc[tradingDayIndex] = cash.cashinhand.iloc[tradingDayIndex] - (priceForTheDay * float(numShares))
                    print("Date: ", tradingOrder,
                          "Symbol: ", toBuySymbol,
                          "Order: ", ordersdf.ordertype.iloc[index],
                          "Shares: ", numShares,
                          "Price: ", priceForTheDay,
                          "Cash: ", cash.cashinhand.iloc[tradingDayIndex] )
                elif ordersdf.ordertype.iloc[index] == 'Sell':
                    toSellSymbol = ordersdf.symbol.iloc[index]
                    toSell = symbols.index(toSellSymbol)
                    numShares = ordersdf.volume.iloc[index]
                    priceForTheDay = closingPrices.iloc[tradingDayIndex][toSell]
                    cash.cashinhand.iloc[tradingDayIndex] = cash.cashinhand.iloc[tradingDayIndex] + (priceForTheDay * float(numShares))
                    print("Date: ", tradingOrder,
                          "Symbol: ", toSellSymbol,
                          "Order: ", ordersdf.ordertype.iloc[index],
                          "Shares: ", numShares,
                          "Price: ", priceForTheDay,
                          "Cash: ", cash.cashinhand.iloc[tradingDayIndex] )
                else:
                    print("error")
                index+=1
        

        valueFrame['valueOfPortfolio'] = cash['cashinhand'].values
        
    valueFrame.index = closingPrices.index

    return valueFrame

### Write values to file

### Define market simulation function

In [6]:



initialCash = 1000000
ordersFilename = 'input/orders.csv'
valuesFilename = 'output/values.csv'

# Reading the data from the file, and getting a NumPy matrix
ordersDataFrame, symbols = readOrdersFileIntoDF(ordersFilename)

# Getting data from marketsim
print("List of transactions")
valueFrame = marketsim(initialCash, ordersDataFrame, symbols)

# Writing data to file
writeValuesIntoCSV(valuesFilename, valueFrame)

List of transactions
2011-01-10 00:00:00 2011-12-20 00:00:00
['IBM', 'AAPL', 'GOOG', 'XOM']
[*********************100%***********************]  4 of 4 downloaded
Date:  2011-01-10 00:00:00 Symbol:  AAPL Order:  Buy Shares:  1500 Price:  43.356891999999995 Cash:  934964.662
Date:  2011-01-13 00:00:00 Symbol:  AAPL Order:  Sell Shares:  1500 Price:  43.765831 Cash:  1000613.4085
Date:  2011-01-13 00:00:00 Symbol:  IBM Order:  Buy Shares:  4000 Price:  117.880554 Cash:  529091.1925
Date:  2011-01-26 00:00:00 Symbol:  GOOG Order:  Buy Shares:  1000 Price:  306.258087 Cash:  222833.1055
Date:  2011-02-02 00:00:00 Symbol:  XOM Order:  Sell Shares:  4000 Price:  65.633003 Cash:  485365.1175
Date:  2011-02-10 00:00:00 Symbol:  XOM Order:  Buy Shares:  4000 Price:  65.81276700000001 Cash:  222114.04949999996
Date:  2011-03-03 00:00:00 Symbol:  GOOG Order:  Sell Shares:  1000 Price:  302.810516 Cash:  524924.5655
Date:  2011-03-03 00:00:00 Symbol:  IBM Order:  Sell Shares:  2200 Price:  130.0054

## Part 2: Part 2: Transaction Costs
